In [8]:
import os
import import_ipynb
import sys
from basic_check import check
from json_check import check_json
from jsonld_check import check_jsonld

importing Jupyter notebook from basic_check.ipynb


In [11]:
def validate(tar_file):
    
    check_result = False
    size_size = 0
    extension = os.path.splitext(tar_file)[1]
    
    c = check(tar_file, extension)
    check_result = c.existence_check(tar_file)
    if check_result == False:
        print("There is no file or directory called %s" % tar_file, " It does exist or is not udner current directory.")
        raise Exception("The file exitence check -- FAIL")
    else:
        size_check = c.file_size_check(tar_file)
        if size_check == 0:
            print("The size of RO-Crate cannot be zero byte.")
            raise Exception("This is an invalid RO-Crate.")
        else:
            check_result = c.metadata_check(tar_file, extension)
            if check_result == False:
                raise Exception("NO Metadata File, Invalid RO-Crate.")
            else:
                js = check_json(tar_file, extension)
                check_result = js.string_value_check(tar_file, extension)
                
                if isinstance(check_result, list):
                    error_message = js.string_value_check(tar_file, extension)[1]
                    raise Exception(error_message)
                else:
                    jsld = check_jsonld(tar_file, extension)
                    check_result = jsld.check_context(tar_file, extension)
                    if check_result == False:
                        raise Exception("NO context provided, Invalid RO-Crate with incorrect Json-ld format.")
    
    print("This is a valid RO-Crate.")
                    
        
    
    